# Evolver Loop 2 Analysis

Systematically scan all snapshots to find the best per-N configurations.
This is a comprehensive scan of ~100 snapshot folders.

In [1]:
import pandas as pd
import numpy as np
import math
from numba import njit
import os
from glob import glob
import json

# Tree vertices
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

In [2]:
@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a single N configuration"""
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx = min(mnx, X)
            mxx = max(mxx, X)
            mny = min(mny, Y)
            mxy = max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

In [3]:
def parse_submission(filepath):
    """Parse submission CSV and return dict of N -> (xs, ys, degs)"""
    try:
        df = pd.read_csv(filepath)
        if len(df) == 0:
            return None, None
        
        # Parse values (remove 's' prefix)
        df['x_val'] = df['x'].str.replace('s', '').astype(float)
        df['y_val'] = df['y'].str.replace('s', '').astype(float)
        df['deg_val'] = df['deg'].str.replace('s', '').astype(float)
        
        # Extract N from id (format: NNN_idx)
        df['N'] = df['id'].str.split('_').str[0].astype(int)
        
        configs = {}
        for n, group in df.groupby('N'):
            xs = group['x_val'].values
            ys = group['y_val'].values
            degs = group['deg_val'].values
            configs[n] = (xs, ys, degs)
        
        return configs, df
    except Exception as e:
        return None, None

In [4]:
def calculate_scores_by_n(configs, tx, ty):
    """Calculate scores for each N value"""
    scores_by_n = {}
    for n in range(1, 201):
        if n in configs:
            xs, ys, degs = configs[n]
            score = score_group(xs, ys, degs, tx, ty)
            scores_by_n[n] = score
    return scores_by_n

In [5]:
# Find all submission CSVs in snapshots
snapshot_base = '/home/nonroot/snapshots/santa-2025'
snapshot_dirs = sorted(glob(f'{snapshot_base}/*/'))

print(f'Found {len(snapshot_dirs)} snapshot directories')

# Collect all submission files
all_submissions = []

# Check preoptimized folder
preopt_files = [
    '/home/code/preoptimized/submission.csv',
    '/home/code/preoptimized/best_snapshot.csv',
    '/home/code/preoptimized/saspav_best.csv',
    '/home/code/preoptimized/smartmanoj_submission.csv',
    '/home/code/preoptimized/ensemble.csv',
]
for f in preopt_files:
    if os.path.exists(f):
        all_submissions.append(('preopt', f))

# Check each snapshot
for snap_dir in snapshot_dirs:
    snap_id = os.path.basename(snap_dir.rstrip('/'))
    # Check common locations for submission files
    possible_paths = [
        f'{snap_dir}submission/submission.csv',
        f'{snap_dir}code/submission.csv',
        f'{snap_dir}submission.csv',
    ]
    for path in possible_paths:
        if os.path.exists(path):
            all_submissions.append((snap_id, path))
            break

print(f'Found {len(all_submissions)} submission files to scan')

Found 100 snapshot directories
Found 83 submission files to scan


In [6]:
# Score all submissions and track best per-N
best_scores_by_n = {n: float('inf') for n in range(1, 201)}
best_source_by_n = {n: None for n in range(1, 201)}
best_configs_by_n = {n: None for n in range(1, 201)}

all_totals = {}

for source_id, filepath in all_submissions:
    configs, df = parse_submission(filepath)
    if configs is None:
        continue
    
    scores_by_n = calculate_scores_by_n(configs, TX, TY)
    total = sum(scores_by_n.values())
    all_totals[source_id] = total
    
    # Update best per-N
    for n in range(1, 201):
        if n in scores_by_n and scores_by_n[n] < best_scores_by_n[n]:
            best_scores_by_n[n] = scores_by_n[n]
            best_source_by_n[n] = source_id
            best_configs_by_n[n] = configs[n]

print(f'Scanned {len(all_totals)} valid submissions')

Scanned 78 valid submissions


In [7]:
# Show top 10 submissions by total score
sorted_totals = sorted(all_totals.items(), key=lambda x: x[1])
print('\nTop 10 submissions by total score:')
for source_id, total in sorted_totals[:10]:
    print(f'  {source_id}: {total:.6f}')


Top 10 submissions by total score:
  21328309666: 27.414787
  21328310479: 70.523320
  21145966992: 70.572798
  21322576827: 70.616145
  21322576451: 70.619825
  21198925328: 70.624381
  21198927060: 70.624381
  21222373488: 70.624381
  21222375510: 70.624381
  21222377956: 70.624381


In [8]:
# Calculate optimal ensemble score
ensemble_total = sum(best_scores_by_n.values())
print(f'\nOptimal ensemble from ALL sources: {ensemble_total:.6f}')
print(f'Best single submission: {sorted_totals[0][1]:.6f}')
print(f'Improvement from ensemble: {sorted_totals[0][1] - ensemble_total:.6f}')


Optimal ensemble from ALL sources: 27.414787
Best single submission: 27.414787
Improvement from ensemble: 0.000000


In [9]:
# Show source breakdown
source_counts = {}
for n in range(1, 201):
    source = best_source_by_n[n]
    source_counts[source] = source_counts.get(source, 0) + 1

print('\nSource breakdown for optimal ensemble:')
for source, count in sorted(source_counts.items(), key=lambda x: -x[1]):
    print(f'  {source}: {count} N values')


Source breakdown for optimal ensemble:
  21328309666: 199 N values
  preopt: 1 N values


In [10]:
# Show which N values improved vs best_snapshot
best_snapshot_configs, _ = parse_submission('/home/code/preoptimized/best_snapshot.csv')
best_snapshot_scores = calculate_scores_by_n(best_snapshot_configs, TX, TY)

print('\nN values where ensemble beats best_snapshot:')
improvements = []
for n in range(1, 201):
    if n in best_snapshot_scores:
        improvement = best_snapshot_scores[n] - best_scores_by_n[n]
        if improvement > 1e-9:
            improvements.append((n, improvement, best_source_by_n[n]))

if improvements:
    for n, imp, source in sorted(improvements, key=lambda x: -x[1])[:20]:
        print(f'  N={n}: improvement {imp:.6f} from {source}')
else:
    print('  No improvements found - best_snapshot already optimal')


N values where ensemble beats best_snapshot:
  N=24: improvement 0.348369 from 21328309666
  N=41: improvement 0.337622 from 21328309666
  N=31: improvement 0.336179 from 21328309666
  N=33: improvement 0.335999 from 21328309666
  N=46: improvement 0.335351 from 21328309666
  N=44: improvement 0.334852 from 21328309666
  N=49: improvement 0.332666 from 21328309666
  N=38: improvement 0.330715 from 21328309666
  N=45: improvement 0.329825 from 21328309666
  N=42: improvement 0.328932 from 21328309666
  N=35: improvement 0.326609 from 21328309666
  N=50: improvement 0.326466 from 21328309666
  N=48: improvement 0.325896 from 21328309666
  N=47: improvement 0.325391 from 21328309666
  N=34: improvement 0.325215 from 21328309666
  N=40: improvement 0.324974 from 21328309666
  N=39: improvement 0.324206 from 21328309666
  N=43: improvement 0.322050 from 21328309666
  N=29: improvement 0.320657 from 21328309666
  N=32: improvement 0.320296 from 21328309666


In [11]:
# Save the optimal ensemble
def generate_submission(configs_by_n, output_path):
    rows = []
    for n in range(1, 201):
        xs, ys, degs = configs_by_n[n]
        for i in range(len(xs)):
            row = {
                'id': f'{n:03d}_{i}',
                'x': f's{xs[i]}',
                'y': f's{ys[i]}',
                'deg': f's{degs[i]}'
            }
            rows.append(row)
    df = pd.DataFrame(rows)
    df.to_csv(output_path, index=False)
    return df

# Only save if we found improvements
if ensemble_total < sorted_totals[0][1] - 1e-9:
    os.makedirs('/home/submission', exist_ok=True)
    submission_df = generate_submission(best_configs_by_n, '/home/submission/submission.csv')
    print(f'\nSaved optimal ensemble to /home/submission/submission.csv')
    print(f'Total rows: {len(submission_df)}')
    print(f'Score: {ensemble_total:.6f}')
else:
    print(f'\nNo improvement over best single submission - keeping best_snapshot')
    # Copy best_snapshot to submission
    import shutil
    os.makedirs('/home/submission', exist_ok=True)
    shutil.copy('/home/code/preoptimized/best_snapshot.csv', '/home/submission/submission.csv')
    print(f'Copied best_snapshot.csv to /home/submission/submission.csv')


No improvement over best single submission - keeping best_snapshot
Copied best_snapshot.csv to /home/submission/submission.csv


In [13]:
# Check if the 21328309666 submission has overlapping trees
# This is suspicious because it scores 27.4 which is way below target

# Simple check: count how many trees have identical coordinates
suspicious_path = '/home/nonroot/snapshots/santa-2025/21328309666/submission/submission.csv'
df = pd.read_csv(suspicious_path)
df['x_val'] = df['x'].str.replace('s', '').astype(float)
df['y_val'] = df['y'].str.replace('s', '').astype(float)
df['deg_val'] = df['deg'].str.replace('s', '').astype(float)
df['N'] = df['id'].str.split('_').str[0].astype(int)

# Check for duplicate positions within each N
print("Checking 21328309666 for duplicate tree positions:")
invalid_ns = []
for n, group in df.groupby('N'):
    # Check if any trees have identical x,y,deg
    coords = group[['x_val', 'y_val', 'deg_val']].values
    unique_coords = np.unique(coords, axis=0)
    if len(unique_coords) < len(coords):
        invalid_ns.append(n)

print(f"  N values with duplicate tree positions: {len(invalid_ns)}")
print(f"  Examples: {invalid_ns[:20]}")

# This submission is INVALID - trees are overlapping
print("\n*** This submission is INVALID - cannot be used ***")

Checking 21328309666 for duplicate tree positions:
  N values with duplicate tree positions: 49
  Examples: [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28]

*** This submission is INVALID - cannot be used ***


In [14]:
# Filter out invalid submissions and find the true best
# The second-best was 21328310479 at 70.523320 - let's verify it's valid

def check_submission_validity(filepath):
    """Check if submission has duplicate tree positions"""
    try:
        df = pd.read_csv(filepath)
        df['x_val'] = df['x'].str.replace('s', '').astype(float)
        df['y_val'] = df['y'].str.replace('s', '').astype(float)
        df['deg_val'] = df['deg'].str.replace('s', '').astype(float)
        df['N'] = df['id'].str.split('_').str[0].astype(int)
        
        for n, group in df.groupby('N'):
            coords = group[['x_val', 'y_val', 'deg_val']].values
            unique_coords = np.unique(coords, axis=0)
            if len(unique_coords) < len(coords):
                return False
        return True
    except:
        return False

# Check top submissions for validity
print("Checking validity of top submissions:")
valid_submissions = []
for source_id, total in sorted_totals[:20]:
    # Find the filepath
    for sid, fpath in all_submissions:
        if sid == source_id:
            is_valid = check_submission_validity(fpath)
            status = "VALID" if is_valid else "INVALID"
            print(f"  {source_id}: {total:.6f} - {status}")
            if is_valid:
                valid_submissions.append((source_id, total, fpath))
            break

print(f"\nBest VALID submission: {valid_submissions[0][0]} with score {valid_submissions[0][1]:.6f}")

Checking validity of top submissions:
  21328309666: 27.414787 - INVALID
  21328310479: 70.523320 - VALID


  21145966992: 70.572798 - VALID
  21322576827: 70.616145 - VALID


  21322576451: 70.619825 - VALID
  21198925328: 70.624381 - VALID
  21198927060: 70.624381 - VALID


  21222373488: 70.624381 - VALID
  21222375510: 70.624381 - VALID
  21222377956: 70.624381 - VALID


  21222390477: 70.624381 - VALID
  21328309254: 70.624381 - VALID


  21322577324: 70.625376 - VALID
  21191209482: 70.625918 - VALID
  21198893057: 70.625918 - VALID


  21198928571: 70.625918 - VALID
  21328310048: 70.625918 - VALID


  21222392487: 70.626088 - VALID
  21191211160: 70.627582 - VALID
  21198790429: 70.627582 - VALID

Best VALID submission: 21328310479 with score 70.523320


In [15]:
# Create optimal ensemble from VALID submissions only
print("Creating optimal ensemble from VALID submissions only...")

# Re-scan with validity check
best_scores_by_n_valid = {n: float('inf') for n in range(1, 201)}
best_source_by_n_valid = {n: None for n in range(1, 201)}
best_configs_by_n_valid = {n: None for n in range(1, 201)}

for source_id, filepath in all_submissions:
    if not check_submission_validity(filepath):
        continue
    
    configs, df = parse_submission(filepath)
    if configs is None:
        continue
    
    scores_by_n = calculate_scores_by_n(configs, TX, TY)
    
    # Update best per-N
    for n in range(1, 201):
        if n in scores_by_n and scores_by_n[n] < best_scores_by_n_valid[n]:
            best_scores_by_n_valid[n] = scores_by_n[n]
            best_source_by_n_valid[n] = source_id
            best_configs_by_n_valid[n] = configs[n]

# Calculate ensemble score
ensemble_total_valid = sum(best_scores_by_n_valid.values())
print(f"\nOptimal VALID ensemble score: {ensemble_total_valid:.6f}")
print(f"Best single VALID submission: 70.523320")
print(f"Previous best (best_snapshot): 70.624381")
print(f"Improvement over best_snapshot: {70.624381 - ensemble_total_valid:.6f}")

Creating optimal ensemble from VALID submissions only...



Optimal VALID ensemble score: 70.523320
Best single VALID submission: 70.523320
Previous best (best_snapshot): 70.624381
Improvement over best_snapshot: 0.101061


In [ ]:
# Show source breakdown for valid ensemble
source_counts_valid = {}
for n in range(1, 201):
    source = best_source_by_n_valid[n]
    source_counts_valid[source] = source_counts_valid.get(source, 0) + 1

print('Source breakdown for VALID optimal ensemble:')
for source, count in sorted(source_counts_valid.items(), key=lambda x: -x[1]):
    print(f'  {source}: {count} N values')

# Show which N values improved vs best_snapshot
print('\nTop 20 N values where valid ensemble beats best_snapshot:')
improvements_valid = []
for n in range(1, 201):
    if n in best_snapshot_scores:
        improvement = best_snapshot_scores[n] - best_scores_by_n_valid[n]
        if improvement > 1e-9:
            improvements_valid.append((n, improvement, best_source_by_n_valid[n]))

for n, imp, source in sorted(improvements_valid, key=lambda x: -x[1])[:20]:
    print(f'  N={n}: improvement {imp:.6f} from {source}')

In [ ]:
# Verify the submission
verify_configs, _ = parse_submission('/home/submission/submission.csv')
verify_scores = calculate_scores_by_n(verify_configs, TX, TY)
verify_total = sum(verify_scores.values())
print(f'\nVerification of saved submission: {verify_total:.6f}')